In [13]:
import sys
import os
import asyncio
root = r"d:\Projects\University\Semester06\cs221-nlp"
if root not in sys.path:
    sys.path.append(root)

In [7]:
from src.agents.database_retriever import DatabaseRetriever
import pandas

In [5]:
retriever = DatabaseRetriever.from_config(config_path='../../configs/indexing_pipeline.yml')

-> Loading config từ ../../configs/indexing_pipeline.yml...
-> Khôi phục lại Embedding Model: bkai-foundation-models/vietnamese-bi-encoder trên cuda...
-> Kết nối vào Vector DB tại: ../../data/legal_documents/chroma_db (Collection: vn_law)
-> Khởi tạo BM25Retriever thành công (3222 docs).
-> Đã loại bỏ EnsembleRetriever (RRF).
-> DatabaseRetriever đã được khởi tạo thành công (Chế độ BM25 + Vector Riêng biệt).


# 1. Dataset

In [9]:
df = pandas.read_csv(
    r'../../data/question_answer/test.csv'
)
df

,topic,question,answer
0,THỦ TỤC BỔ NHIỆM GIÁM ĐỊNH VIÊN PHÁP Y VÀ GIÁM...,Hồ sơ đề nghị bổ nhiệm gồm những giấy tờ nào,Hồ sơ bổ nhiệm Giám định viên pháp y và Giám đ...
1,THỦ TỤC HÀNH CHÍNH LĨNH VỰC BẢO HIỂM Y TẾ,Nhà nước có chính sách gì đối với BHYT?,1. Nhà nước đóng hoặc hỗ trợ tiền đóng BHYT ch...
2,PHẠM VI ĐƯỢC HƯỞNG BẢO HIỂM Y TẾ,Người tham gia BHYT đi khám chữa bệnh tại các ...,Người tham gia BHYT khi đi khám chữa bệnh tại ...
3,TRANG THIẾT BỊ VÀ SINH PHẨM Y TẾ,"Khoản 5, Điều 66 của Nghị định 36/2016/NĐ-CP q...",Công ty đã công bố đủ điều kiện phân loại trướ...
4,VỀ AN TOÀN SINH HỌC,Điều kiện của cơ sở an toàn sinh học cấp I là gì?,Điều kiện của cơ sở an toàn sinh học cấp I the...
5,PHẠM VI ĐƯỢC HƯỞNG BẢO HIỂM Y TẾ,Như thế nào là đi khám chữa bệnh BHYT đúng quy...,"Đi khám, chữa bệnh đúng quy định (trừ trường h..."
6,THỦ TỤC HÀNH CHÍNH LĨNH VỰC DƯỢC,Quầy thuốc của tôi được cấp GPP từ năm 2016 và...,Hoạt chất ofloxacin nằm trong danh mục dược ch...
7,"ĐỐI TƯỢNG, MỨC ĐÓNG, TRÁCH NHIỆM VÀ PHƯƠNG THỨ...",Trường hợp một người đồng thời thuộc nhiều đối...,Trường hợp một người đồng thời thuộc nhiều đối...
8,CẤP PHIẾU TIẾP NHẬN BẢN CÔNG BỐ DƯỢC LIỆU SẢN ...,Trình tự thực hiện thủ tục Cấp Phiếu tiếp nhận...,3 bước
9,"TỔ CHỨC KHÁM BỆNH, CHỮA BỆNH BẢO HIỂM Y TẾ VÀ ...","Hợp đồng khám bệnh,chữa bệnh BHYT là gì ?","Hợp đồng khám bệnh, chữa bệnh BHYT là văn bản ..."


In [14]:
def run_retrieve(q):
    return asyncio.get_event_loop().run_until_complete(retriever.retrieve(q, k=3))

In [21]:
tasks = [retriever.retrieve(q, k=3) for q in df['question']]
all_contexts = await asyncio.gather(*tasks)


[QUERY]: Hồ sơ đề nghị bổ nhiệm gồm những giấy tờ nào
-> BM25 Search tìm thấy 10 đoạn.
-> Vector Search tìm thấy 1 đoạn.
-> Hợp nhất thủ công (Loại trùng lặp) còn 11 đoạn.
-> Trả về 3 đoạn cuối cùng (top-k của tập hợp nhất).
   1. [Thông tư 41/2011/TT-BYT]: c) Trường hợp hồ sơ đề nghị cấp, cấp lại, điều chỉnh giấy phép hoạt động chưa hợ...
   2. [Thông tư 02/2024/TT-BYT]: Tôi xin gửi kèm theo đơn này bộ hồ sơ gồm các giấy tờ sau:5 (1) ...................
   3. [Thông tư 02/2024/TT-BYT]: PHIẾU TIẾP NHẬN Hồ sơ đề nghị cấp/cấp lại giấy chứng nhận lương y Họ và tên ngườ...

[QUERY]: Nhà nước có chính sách gì đối với BHYT?
-> BM25 Search tìm thấy 10 đoạn.
-> Vector Search tìm thấy 1 đoạn.
-> Hợp nhất thủ công (Loại trùng lặp) còn 11 đoạn.
-> Trả về 3 đoạn cuối cùng (top-k của tập hợp nhất).
   1. [Nghị định 85/2012/NĐ-CP]: Bộ Y tế chủ trì, phối hợp với Bộ Tài chính ban hành mức chi khám bệnh, chữa bệnh...
   2. [Nghị định 85/2012/NĐ-CP]: Điều 23. Nguồn đảm bảo chính sách tiền lương 1. 

In [23]:
df['context'] = all_contexts
df['context_str'] = df['context'].apply(lambda docs: "\n".join([d.page_content for d in docs]))
df.drop(columns=['context_str'], inplace=True)
df.head()

,topic,question,answer,context
0,THỦ TỤC BỔ NHIỆM GIÁM ĐỊNH VIÊN PHÁP Y VÀ GIÁM...,Hồ sơ đề nghị bổ nhiệm gồm những giấy tờ nào,Hồ sơ bổ nhiệm Giám định viên pháp y và Giám đ...,[page_content='c) Trường hợp hồ sơ đề nghị cấp...
1,THỦ TỤC HÀNH CHÍNH LĨNH VỰC BẢO HIỂM Y TẾ,Nhà nước có chính sách gì đối với BHYT?,1. Nhà nước đóng hoặc hỗ trợ tiền đóng BHYT ch...,"[page_content='Bộ Y tế chủ trì, phối hợp với B..."
2,PHẠM VI ĐƯỢC HƯỞNG BẢO HIỂM Y TẾ,Người tham gia BHYT đi khám chữa bệnh tại các ...,Người tham gia BHYT khi đi khám chữa bệnh tại ...,[page_content='Điều 2. Nguyên tắc chung\n1. Ng...
3,TRANG THIẾT BỊ VÀ SINH PHẨM Y TẾ,"Khoản 5, Điều 66 của Nghị định 36/2016/NĐ-CP q...",Công ty đã công bố đủ điều kiện phân loại trướ...,"[page_content='e) Thanh tra, kiểm tra, giám sá..."
4,VỀ AN TOÀN SINH HỌC,Điều kiện của cơ sở an toàn sinh học cấp I là gì?,Điều kiện của cơ sở an toàn sinh học cấp I the...,[page_content='d) Bảo đảm xử lý rác thải y tế ...


In [24]:
df.to_csv(r'../../data/question_answer/test_with_context.csv', index=False)

In [25]:
print(df.iloc[0]['context'])

[Document(metadata={'status': 'Hết hiệu lực toàn bộ', 'source': 'Thông tư 41/2011/TT-BYT', 'effective_date': '01/01/2012', 'type': 'law_document', 'link': 'https://vbpl.vn/TW/Pages/vbpq-toanvan.aspx?ItemID=46985&Keyword=kh%C3%A1m%20ch%E1%BB%AFa%20b%E1%BB%87nh'}, page_content='c) Trường hợp hồ sơ đề nghị cấp, cấp lại, điều chỉnh giấy phép hoạt động chưa hợp lệ thì trong thời gian 10 ngày làm việc, kể từ ngày ghi trên Phiếu tiếp nhận hồ sơ, cơ quan tiếp nhận hồ sơ phải có văn bản thông báo cho cơ sở khám bệnh, chữa bệnh đề nghị cấp, cấp lại, điều chỉnh giấy phép hoạt động để hoàn chỉnh hồ sơ. Văn bản thông báo phải nêu cụ thể là bổ sung những tài liệu nào, nội dung nào cần sửa đổi;'), Document(metadata={'source': 'Thông tư 02/2024/TT-BYT', 'status': 'Hết hiệu lực một phần', 'effective_date': '12/03/2024', 'type': 'law_document', 'link': 'https://vbpl.vn/TW/Pages/vbpq-toanvan.aspx?ItemID=166665&Keyword=kh%C3%A1m%20ch%E1%BB%AFa%20b%E1%BB%87nh'}, page_content='Tôi xin gửi kèm theo đơn này b